In [1]:
!pip install torch

In [1]:
! pip freeze | grep skimage

In [273]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Activation, LSTM
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from matplotlib import image
from skimage.transform import resize
import numpy as np
import os
import torch

In [174]:
data_path = 'data/train'
df = pd.read_csv("data/train.csv", header=None, sep=';')
df = df[0:3]
df

,0,1,2
0,WIN_20180925_17_08_43_Pro_Left_Swipe_new,Left_Swipe_new,0
1,WIN_20180925_17_18_28_Pro_Left_Swipe_new,Left_Swipe_new,0
2,WIN_20180925_17_18_56_Pro_Left_Swipe_new,Left_Swipe_new,0


In [210]:
def load_images(folder):
    loaded_images = []
    for img in os.listdir(f'./{data_path}/{folder}'):
        img_data = image.imread(f'./{data_path}/{folder}/{img}').astype(np.float32)
        image_resized=resize(img_data,(100, 100))
        loaded_images.append(image_resized)
#     loaded_images = np.concatenate(loaded_images, axis=0 )
    loaded_images = np.stack(loaded_images, axis=0)
    return loaded_images

In [229]:
def prepare_data(df):
    train_df = df.copy()
    Y = to_categorical(train_df[2], 5)
#     X = train_df.drop([0, 1, 2, 3], axis=1)
    X = np.stack(train_df[3], axis=0)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    return X_train, X_test, y_train, y_test

In [230]:
df[3] = df[0].apply(lambda x: load_images(x))
# for i in range(29):
#     df[f'position_{i+1}'] = df[3].apply(lambda x: x[i])

In [231]:
X_train, X_test, y_train, y_test = prepare_data(df)

In [232]:
print("X type: ", type(X_train))
print("y type: ", type(y_train))
print("X shape: ", X_train.shape)
print("y shape: ", y_train.shape)

X type:  <class 'numpy.ndarray'>
y type:  <class 'numpy.ndarray'>
X shape:  (2, 30, 100, 100, 3)
y shape:  (2, 5)


In [234]:
type(X_train)

numpy.ndarray

In [290]:
def build_model():
    model = Sequential()
    model.add(Conv3D(16, (3, 3, 3), padding='same',
             input_shape=(30,100, 100, 3
             )))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(Conv3D(32, (3, 3, 3), padding='same',
             input_shape=(15,50, 50, 3
             )))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(5,activation='relu'))
    model.summary()
    return model

model = build_model()

Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_53 (Conv3D)          (None, 30, 100, 100, 16)  1312      
                                                                 
 activation_46 (Activation)  (None, 30, 100, 100, 16)  0         
                                                                 
 batch_normalization_52 (Bat  (None, 30, 100, 100, 16)  64       
 chNormalization)                                                
                                                                 
 max_pooling3d_37 (MaxPoolin  (None, 15, 50, 50, 16)   0         
 g3D)                                                            
                                                                 
 conv3d_54 (Conv3D)          (None, 15, 50, 50, 32)    13856     
                                                                 
 activation_47 (Activation)  (None, 15, 50, 50, 32)  

In [291]:
def compile_model(model):
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

def train_model(model, X_train, Y_train, X_val, Y_val):
    batch_size = 128
    epochs = 12
    history = model.fit(
        X_train, Y_train, 
        batch_size=batch_size, epochs=epochs, 
        verbose=1, validation_data=(X_val, Y_val)
        )
    return model, history


model = compile_model(model)
model, history = train_model(model, X_train, y_train, X_test, y_test)

Epoch 1/12
1/1 [==============================] - 2s 2s/step - loss: 16.1181 - accuracy: 0.0000e+00 - val_loss: 16.1181 - val_accuracy: 0.0000e+00
Epoch 2/12
1/1 [==============================] - 1s 1s/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 1.0000
Epoch 3/12
1/1 [==============================] - 1s 1s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 4/12
1/1 [==============================] - 1s 1s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 5/12
1/1 [==============================] - 1s 1s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 6/12
1/1 [==============================] - 1s 1s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 7/12
1/1 [==============================] - 1s 1s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 8/12
1/1 [==============================] - 1s 1s/step - loss